<a href="https://colab.research.google.com/github/cdr0101/SamplingAssn102117161/blob/main/SamplingAssn102117161.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SAMPLING ASSIGNMENT**

**Submitted by: CHELSI**

**Roll no.: 102117161**

**Group: 3CS6**

Importing the required libraries

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, KFold
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

Loading the data and splitting it after balancing using SMOTE technique.

In [27]:
# Loading dataset
data_df = pd.read_csv('/content/Creditcard_data.csv')

# Separating features (to X) and target (to y)
X = data_df.drop('Class', axis=1)
y = data_df['Class']

# Applying smote technique to balance the data
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

# Spliting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=0)

Defining the models and sampling techniques to be used

In [28]:
models = {
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(),
    'Logistic Regression': LogisticRegression(),
    'KNN': KNeighborsClassifier()
}

sampling_techniques = {
    'Simple Random': 'simpleRandom',
    'Systematic': 'systematic',
    'Stratified': 'stratified',
    'Cluster': 'cluster',
    'Cross Validation': 'crossValidation'
}

Implementing sampling and fitting the models to each sample

In [29]:
results = {}
for model_name, model in models.items():
    model_results = []
    for technique_name, technique in sampling_techniques.items():
        # Sampling the data based on technique to be used
        if technique == 'simpleRandom':
            X_sampled, y_sampled = X_train, y_train
        elif technique == 'systematic':
            # Implementing systematic sampling by selecting every nth item
            n = 5
            X_sampled, _, y_sampled, _ = train_test_split(X_train, y_train, test_size=1-(1/n))
        elif technique == 'stratified': # stratified sampling
            strat_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
            for train_index, _ in strat_split.split(X_train, y_train):
                X_sampled, y_sampled = X_train.iloc[train_index], y_train.iloc[train_index]
        elif technique == 'cluster':
            # Implementing cluster sampling
            # Assuming each row represents a cluster, sample clusters instead of individual data points
            cluster_indices = [i for i in range(0, len(X_train), 50)]  # Assuming clusters of 50 rows each
            X_sampled = X_train.iloc[cluster_indices]
            y_sampled = y_train.iloc[cluster_indices]
        elif technique == 'crossValidation': # Cross validation sampling
            kf = KFold(n_splits=5, random_state=0, shuffle=True)
            for _, test_index in kf.split(X_train, y_train):
                X_sampled, y_sampled = X_train.iloc[test_index], y_train.iloc[test_index]

        model.fit(X_sampled, y_sampled)

        y_pred = model.predict(X_test) # Making predictions
        accuracy = accuracy_score(y_test, y_pred)
        model_results.append(accuracy*100)

    results[model_name] = model_results

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

**Accuracy Table**

In [30]:
results_df = pd.DataFrame(results, index=sampling_techniques.keys()).transpose()
print(results_df)

                     Simple Random  Systematic  Stratified    Cluster  \
Decision Tree            98.366013   90.196078   98.039216  78.758170   
Random Forest            99.019608   96.405229   99.346405  80.392157   
XGBoost                  98.692810   92.156863   98.366013  75.163399   
Logistic Regression      90.522876   89.542484   89.542484  73.856209   
KNN                      83.333333   70.588235   81.699346  54.248366   

                     Cross Validation  
Decision Tree               88.562092  
Random Forest               98.039216  
XGBoost                     93.464052  
Logistic Regression         88.888889  
KNN                         68.300654  


**Observation: Maximum accuracy has been achieved corresponding to STRATIFIED SAMPLING AND RANDOM FOREST MODEL. (99.346405%)**